In [ ]:
from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain.agents import Tool

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
load_dotenv(override=True)

In [ ]:
class State(TypedDict):
    
    messages: Annotated[list, add_messages]
    flagged_ingredients: List[str]
    safety_score: int


graph_builder = StateGraph(State)

In [ ]:


def flag_ingredient(ingredient_name: str, reason: str):
    """Flags a harmful ingredient and records the scientific reason.
        Use this to display warnings on the user's dashboard."""
    return f"FLAGGED: {ingredient_name}. REASON: {reason}"

tool_flag = Tool(
        name="flag_harmful_ingredient",
        func=flag_ingredient,
        description="Useful for marking specific ingredients as dangerous to be displayed on the UI."
    )

In [ ]:
# Introducing nest_asyncio
# Python async code only allows for one "event loop" processing aynchronous events.
# The `nest_asyncio` library patches this, and is used for special situations, if you need to run a nested event loop.

import nest_asyncio
nest_asyncio.apply()

In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

# If you get a NotImplementedError here or later, see the Heads Up at the top of the notebook

async_browser =  create_async_playwright_browser(headless=False)  # headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [ ]:
for tool in tools:
    print(f"{tool.name}={tool}")

In [ ]:

target_url = "https://world.openfoodfacts.org/product/7501008066386/froot-loops-kellogg-s"


tool_dict = {tool.name: tool for tool in tools}
navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool = tool_dict.get("extract_text")


print(f"🕵️ GutCheck is analyzing: {target_url}")
await navigate_tool.arun({"url": target_url})


product_text = await extract_text_tool.arun({})


print("✅ Raw Data Captured.")
print(f"Sample Text: {product_text[:200]}...")

In [ ]:
import textwrap
print(textwrap.fill(product_text))

In [ ]:
all_tools = tools + [tool_push]